# Lesson 3: Chatbot Example

In this lesson, you will familiarize yourself with the chatbot example you will work on during this course. The example includes the tool definitions and execution, as well as the chatbot code. Make sure to interact with the chatbot at the end of this notebook.

## Import Libraries

In [1]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic

## Tool Functions

In [2]:
PAPER_DIR = "papers"

The first tool searches for relevant arXiv papers based on a topic and stores the papers' info in a JSON file (title, authors, summary, paper url and the publication date). The JSON files are organized by topics in the `papers` directory. The tool does not download the papers.  

In [3]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

In [4]:
search_papers("computers")

Results are saved in: papers/computers/papers_info.json


['1312.3300v1', '2207.05241v1', '2012.10468v1', '2009.00041v1', '2009.08005v1']

The second tool looks for information about a specific paper across all topic directories inside the `papers` directory.

In [5]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."

In [6]:
extract_info('1312.3300v1')

'{\n  "title": "Numerical Reproducibility and Parallel Computations: Issues for Interval Algorithms",\n  "authors": [\n    "Nathalie Revol",\n    "Philippe Th\\u00e9veny"\n  ],\n  "summary": "What is called \\"numerical reproducibility\\" is the problem of getting the same result when the scientific computation is run several times, either on the same machine or on different machines, with different types and numbers of processing units, execution environments, computational loads etc. This problem is especially stringent for HPC numerical simulations. In what follows, the focus is on parallel implementations of interval arithmetic using floating-point arithmetic. For interval computations, numerical reproducibility is of course an issue for testing and debugging purposes. However, as long as the computed result encloses the exact and unknown result, the inclusion property, which is the main property of interval arithmetic, is satisfied and getting bit for bit identical results may not

## Tool Schema

Here are the schema of each tool which you will provide to the LLM.

In [7]:
tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arXiv based on a topic and store their information.",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for"
                }, 
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to look for"
                }
            },
            "required": ["paper_id"]
        }
    }
]

## Tool Mapping

This code handles tool mapping and execution.

In [8]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."
        
    elif isinstance(result, list):
        result = ', '.join(result)
        
    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)
    
    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot Code

The chatbot handles the user's queries one by one, but it does not persist memory across the queries.

In [9]:
load_dotenv() 
client = anthropic.Anthropic()

### Query Processing

In [10]:
def process_query(query):
    
    messages = [{'role': 'user', 'content': query}]
    
    response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-sonnet-4-5-20250929', 
                                  tools = tools,
                                  messages = messages)
    
    while response.stop_reason == 'tool_use':
        # Collect all content (text and tool_use blocks)
        assistant_content = []
        tool_results = []

        for content in response.content:
            if content.type == 'text':
                print(content.text)
                assistant_content.append(content)
            elif content.type == 'tool_use':
                assistant_content.append(content)
                
                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                result = execute_tool(tool_name, tool_args)
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": tool_id,
                    "content": result
                })
        
        # Add the assistant message with ALL tool_use blocks
        messages.append({'role': 'assistant', 'content': assistant_content})
        
        # Add the user message with ALL tool_result blocks
        messages.append({"role": "user", "content": tool_results})
        
        # Get the next response
        response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-sonnet-4-5-20250929', 
                                  tools = tools,
                                  messages = messages)
    
    # Print final text response
    for content in response.content:
        if content.type == 'text':
            print(content.text)

### Chat Loop

In [11]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Feel free to interact with the chatbot. Here's an example query: 

- Search for 2 papers on "LLM interpretability"

To access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L3`.

In [12]:
chat_loop()

Type your queries or 'quit' to exit.



Query:  Ayurveda 


I'll search for papers related to Ayurveda on arXiv.
Calling tool search_papers with args {'topic': 'Ayurveda'}
Results are saved in: papers/ayurveda/papers_info.json
I found 5 papers related to Ayurveda. Let me retrieve detailed information about each of them.
Calling tool extract_info with args {'paper_id': '2207.14615v3'}
Calling tool extract_info with args {'paper_id': '2511.02374v1'}
Calling tool extract_info with args {'paper_id': '2412.17005v1'}
Calling tool extract_info with args {'paper_id': '2510.25409v2'}
Calling tool extract_info with args {'paper_id': '2108.09747v1'}
Here are **5 papers related to Ayurveda** found on arXiv:

---

### 1. **Structural investigation of Ayurveda Lauha (Iron) Bhasma**
- **Authors:** M. K. Tiwari, Arjun Singh, Ajay Khooha, U K Goutam
- **Published:** July 29, 2022
- **Summary:** This paper investigates the structural properties of Lauha (Iron) bhasma, an Ayurvedic preparation used to treat iron deficiency. Using X-ray techniques and SEM, researc


Query:  Hypertension


I'll search for papers on hypertension for you.
Calling tool search_papers with args {'topic': 'hypertension'}
Results are saved in: papers/hypertension/papers_info.json
I found 5 papers related to hypertension. Let me retrieve detailed information about each of them.
Calling tool extract_info with args {'paper_id': '2411.11863v1'}
Calling tool extract_info with args {'paper_id': '2111.10471v1'}
Calling tool extract_info with args {'paper_id': '1305.0727v2'}
Calling tool extract_info with args {'paper_id': '2510.19302v2'}
Calling tool extract_info with args {'paper_id': '2007.10717v1'}
Here are 5 recent research papers on hypertension:

## 1. **Longitudinal Wrist PPG Analysis for Reliable Hypertension Risk Screening Using Deep Learning**
**Authors:** Hui Lin, Jiyang Li, Ramy Hussein, et al.  
**Published:** November 2024  
**Summary:** This study uses deep learning models (ResNet and Transformer) to analyze wrist PPG data from smartwatches for hypertension screening. Tested on 448 subj


Query:  quit


<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> To Access the <code>requirements.txt</code> file or the <code>papers</code> folder: </b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em> and finally 3) click on <em>"L3"</em>.
</div>

In the next lessons, you will take out the tool definitions to wrap them in an MCP server. Then you will create an MCP client inside the chatbot to make the chatbot MCP compatible.  

## Resources

[Guide on how to implement tool use](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#how-to-implement-tool-use)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">


<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>